In [2]:
import os.path
import math
import re
import tensorflow as tf
import numpy as np
import facenet
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def get_model_filenames(model_dir):
    files = os.listdir(model_dir)
    meta_files = [s for s in files if s.endswith('.meta')]
    if len(meta_files)==0:
        raise ValueError('No meta file found in the model directory (%s)' % model_dir)
    elif len(meta_files)>1:
        raise ValueError('There should not be more than one meta file in the model directory (%s)' % model_dir)
    meta_file = meta_files[0]
    meta_files = [s for s in files if '.ckpt' in s]
    max_step = -1
    for f in files:
        step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
        if step_str is not None and len(step_str.groups())>=2:
            step = int(step_str.groups()[1])
            if step > max_step:
                max_step = step
                ckpt_file = step_str.groups()[0]
    return meta_file, ckpt_file

def load_model(model, sess):
    model_exp = os.path.expanduser(model)
    print('Model directory: %s' % model_exp)
    meta_file, ckpt_file = get_model_filenames(model_exp)

    print('Metagraph file: %s' % meta_file)
    print('Checkpoint file: %s' % ckpt_file)

    saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file))
    saver.restore(sess, os.path.join(model_exp, ckpt_file))

In [5]:
def forward(paths, batch_size):
    nrof_images = len(paths)
    nrof_batches = int(math.ceil(1.0*nrof_images / batch_size))
    emb_array = np.zeros((nrof_images, embedding_size))

    for i in range(nrof_batches):    
        start_index = i*batch_size
        end_index = min((i+1)*batch_size, nrof_images)
        paths_batch = paths[start_index:end_index]
        images = facenet.load_data(paths_batch, False, False, image_size)
        feed_dict = { images_placeholder:images, phase_train_placeholder:False }
        emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)    
        print("batch number:", i)
    print("Complete")
    
    return emb_array

def printDistance(emb_array, label_list, end=10):
    threshold = 1.0
    lng = len(emb_array[:end])
    print('Distance matrix')
    print('    ', end='')
    for i in range(lng):
        print('    %3d     ' % i, end='')
    print('')
    for i in range(lng):
        print('%1d  ' % i, end='')
        for j in range(lng):
            dist = np.linalg.norm(emb_array[i,:] - emb_array[j,:])
            print('  %1.4f' % dist, end='')
            if (label_list[i] == label_list[j]) and (dist <= threshold):
                print("(TP)", end='')
            elif (label_list[i] != label_list[j]) and (dist > threshold):
                print("(TN)", end='')
            elif (label_list[i] == label_list[j]) and (dist > threshold):
                print("\033[93m(FN)\033[0m", end='')
            elif (label_list[i] != label_list[j]) and (dist <= threshold):
                print("\033[93m(FP)\033[0m", end='')
        print('')
        
def printData(labels, paths, number):
    for i in range(number):
        if i > 0 and labels[i] != labels[i-1]:
            print('')
        print('{0:3d} {1:3d}   {2:s}'.format(i, labels[i], paths[i].split("\\")[1]))
        # print('{0:3d}   {1:s}'.format(labels[i], paths[i].split("\\")[1]))
        
def PCAtransform(emb_array, n_components = 10):
    pca = PCA(n_components, whiten=True)
    pca.fit(emb_array)
    return pca.transform(emb_array), pca

def splitTrainTest(data_set, percent_train):
    paths, labels_list = facenet.get_image_paths_and_labels(data_set)
    
    lng = len(paths)
    lng_train = int(lng * percent_train / 100);
    lng_test = lng - lng_train
    
    train_paths = paths[0:lng_train]
    test_paths = paths[lng_train:]
    
    train_labels = labels_list[0:lng_train]
    test_labels = labels_list[lng_train:]
    
    return train_paths, train_labels, test_paths, test_labels

def trainFisher(emb_array, y):
    emb_del = np.empty_like(emb_array)
    np.copyto(emb_del, emb_array)
    
    emb_array = np.vstack((emb_array, y))
    
    # y = emb_array[sample_number]
    # emb_del = np.delete(emb_array, (sample_number), axis=0)
    
    E = np.linalg.inv(np.cov(emb_array.T))
    D = (y - (1 / (len(emb_array) - 1) * np.sum(emb_del)))
    
    w = np.dot(E, D)
    c = np.dot(w / np.linalg.norm(w), y)
    
    return c, w

def trainFisherWithNum(emb_array, sample_number):    
    y = emb_array[sample_number]
    emb_del = np.delete(emb_array, (sample_number), axis=0)
    
    E = np.linalg.inv(np.cov(emb_array.T))
    D = (y - (1 / (len(emb_array) - 1) * np.sum(emb_del)))
    
    w = np.dot(E, D)
    c = np.dot(w / np.linalg.norm(w), y)
    
    return c, w    

def Fisher(w, c, sample):
    return np.dot(w / np.linalg.norm(w), sample) - c

In [6]:
sess = tf.Session()

In [7]:
data_set = facenet.get_dataset("../../datasets/ownpeople/ownpeople_mtcnnpy_160/")

In [8]:
train_paths, train_labels, test_paths, test_labels = splitTrainTest(data_set, 100)

In [9]:
load_model("20170511-185253", sess)

Model directory: 20170511-185253
Metagraph file: model-20170511-185253.meta
Checkpoint file: model-20170511-185253.ckpt-80000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from 20170511-185253\model-20170511-185253.ckpt-80000


In [11]:
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

image_size = 160
embedding_size = embeddings.get_shape()[1]

In [12]:
emb_array = forward(train_paths, 100) # 100 means 100 mini-batches

batch number: 0
Complete


In [20]:
printDistance(emb_array, train_labels, end=10)

Distance matrix
          0           1           2           3           4           5           6           7           8           9     
0    0.0000(TP)  0.5827(TP)  0.5887(TP)  0.6242(TP)  0.6273(TP)  0.8732(TP)  0.6039(TP)  0.5684(TP)  0.8186(TP)  0.7623(TP)
1    0.5827(TP)  0.0000(TP)  0.5602(TP)  0.6899(TP)  0.7414(TP)  0.8884(TP)  0.6791(TP)  0.6024(TP)  0.7650(TP)  0.6863(TP)
2    0.5887(TP)  0.5602(TP)  0.0000(TP)  0.5301(TP)  0.7963(TP)  0.8490(TP)  0.7058(TP)  0.6925(TP)  0.7630(TP)  0.7677(TP)
3    0.6242(TP)  0.6899(TP)  0.5301(TP)  0.0000(TP)  0.8518(TP)  0.8076(TP)  0.7925(TP)  0.7215(TP)  0.9454(TP)  0.9043(TP)
4    0.6273(TP)  0.7414(TP)  0.7963(TP)  0.8518(TP)  0.0000(TP)  0.8280(TP)  0.6254(TP)  0.7670(TP)  0.8872(TP)  0.7666(TP)
5    0.8732(TP)  0.8884(TP)  0.8490(TP)  0.8076(TP)  0.8280(TP)  0.0000(TP)  1.0181(FN)  0.9651(TP)  0.9104(TP)  0.8550(TP)
6    0.6039(TP)  0.6791(TP)  0.7058(TP)  0.7925(TP)  0.6254(TP)  1.0181(FN)  0.0000(TP)  0.5409(TP)  0.8288(TP)  0.

In [22]:
# Copying for save the original array
emb_origin = np.empty_like(emb_array)
np.copyto(emb_origin, emb_array)

In [34]:
emb_pca, _ = PCAtransform(emb_array, 120)

In [35]:
origin_pca, _ = PCAtransform(emb_origin, 120)

In [50]:
printData(train_labels, train_paths, 23)

  0   1   photo_2016-11-04_19-33-44.png
  1   1   photo_2016-11-19_23-42-00.png
  2   1   photo_2016-12-25_20-59-04.png
  3   1   photo_2016-12-25_20-59-23.png
  4   1   photo_2017-03-10_12-46-19.png
  5   1   photo_2017-05-23_21-38-22.png
  6   1   photo_2017-08-26_20-24-22.png
  7   1   photo_2017-09-08_15-05-01.png
  8   1   photo_2017-10-24_15-01-13.png
  9   1   photo_2017-10-31_16-36-11.png

 10   3   photo_2016-07-04_22-38-35.png
 11   3   photo_2016-07-12_13-26-07.png
 12   3   photo_2016-07-21_00-38-16.png
 13   3   photo_2016-12-12_17-25-22.png
 14   3   photo_2017-01-14_20-14-58.png
 15   3   photo_2017-05-19_00-35-56.png
 16   3   photo_2017-07-15_17-05-07.png
 17   3   photo_2017-07-23_23-45-38.png
 18   3   photo_2017-09-18_12-12-13.png
 19   3   photo_2017-09-26_00-32-31.png
 20   3   photo_2017-11-17_01-42-09.png
 21   3   photo_2017-11-17_01-42-51.png
 22   3   photo_2017-11-17_01-43-40.png


In [67]:
# Cut false positives
emb_del = np.empty_like(emb_array)

In [82]:
arr = emb_array

lng = len(arr)
summ = 0
threshold = 1.05

for i in range(lng):
    for j in range(i, lng):
        dist = np.linalg.norm(arr[i,:] - arr[j,:])
        if (train_labels[i] == train_labels[j]) and (dist > threshold):
            print(i,j,train_labels[i],train_labels[j], dist, "FN")
        elif (train_labels[i] != train_labels[j]) and (dist <= threshold):
            print(i,j,train_labels[i],train_labels[j], dist, "FP")

10 18 3 3 1.1004116607 FN
10 20 3 3 1.0794165457 FN
10 21 3 3 1.07996317147 FN
10 22 3 3 1.08757811171 FN
11 17 3 3 1.05254480804 FN
12 18 3 3 1.0546051841 FN
12 22 3 3 1.08865854368 FN
13 21 3 3 1.08653335662 FN
19 21 3 3 1.0742474607 FN


In [76]:
emb_del = np.delete(emb_array, (10, 11, 13, 19, 22), axis=0)

In [77]:
c, w = trainFisherWithNum(emb_del, 10)

In [81]:
for i in range(len(emb_array)):
    l = Fisher(w, c, emb_array[i])
    print(i, l)

0 -4.16333634234e-17
1 -9.02056207508e-17
2 -1.07552855511e-16
3 -1.2490009027e-16
4 -6.93889390391e-17
5 -1.17961196366e-16
6 -2.77555756156e-17
7 -5.55111512313e-17
8 -6.93889390391e-17
9 -9.71445146547e-17
10 0.0624907122675
11 0.105591563776
12 0.0
13 -0.0117851814857
14 -7.2858385991e-17
15 -3.81639164715e-17
16 -7.6327832943e-17
17 -9.71445146547e-17
18 -2.77555756156e-17
19 -0.0104855840395
20 -3.46944695195e-17
21 -1.38777878078e-17
22 -0.0581120860376
